In [1]:
#!pip install -e git+https://github.com/casperbroch/ai-economist@stockmarket#egg=ai-economist

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import time

In [3]:
# Define the configuration of the environment that will be built

env_config_dict = {
    # ===== SCENARIO CLASS =====
    # Which Scenario class to use: the class's name in the Scenario Registry (foundation.scenarios).
    # The environment object will be an instance of the Scenario class.
    'scenario_name': 'stock_market_simulation',
    
    # ===== COMPONENTS =====
    # Which components to use (specified as list of ("component_name", {component_kwargs}) tuples).
    #   "component_name" refers to the Component class's name in the Component Registry (foundation.components)
    #   {component_kwargs} is a dictionary of kwargs passed to the Component class
    # The order in which components reset, step, and generate obs follows their listed order below.
    'components': [
        # (1) Building houses
        ('BuyOrSellStocks', {}),
        # (2) Trading collectible resources
        #('ExecCircuitBreaker', {}),
    ],
    

    # ===== STANDARD ARGUMENTS ======
    # kwargs that are used by every Scenario class (i.e. defined in BaseEnvironment)
    'n_agents': 4,          # Number of non-planner agents (must be > 1)
    'world_size': [1, 1], # [Height, Width] of the env world
    'episode_length': 100, # Number of timesteps per episode
    
    # In multi-action-mode, the policy selects an action for each action subspace (defined in component code).
    # Otherwise, the policy selects only 1 action.
    'multi_action_mode_agents': False,
    'multi_action_mode_planner': False,
    
    # When flattening observations, concatenate scalar & vector observations before output.
    # Otherwise, return observations with minimal processing.
    'flatten_observations': False,
    # When Flattening masks, concatenate each action subspace mask into a single array.
    # Note: flatten_masks = True is required for masking action logits in the code below.
    'flatten_masks': True,
    'dense_log_frequency': 1
}

In [4]:
from rllib.env_wrapper import RLlibEnvWrapper
env_obj = RLlibEnvWrapper({"env_config_dict": env_config_dict}, verbose=True)

Inside covid19_components.py: 0 GPUs are available.
No GPUs found! Running the simulation on a CPU.
Inside covid19_env.py: 0 GPUs are available.
No GPUs found! Running the simulation on a CPU.


C:\Users\caspe\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\caspe\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\caspe\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\caspe\AppData\Roaming\Python\Python37\site-

std  0.46477668308871967
liquidity 1.0
[EnvWrapper] Spaces
[EnvWrapper] Obs (a)   
BuyOrSellStocks-stock_price: (1,)
action_mask    : (21,)
time           : (1,)
world-Endogenous-AbleToBuy: (1,)
world-Endogenous-AbleToSell: (1,)
world-Endogenous-AvailableFunds: (1,)
world-Endogenous-Demand: (1,)
world-Endogenous-Labor: (1,)
world-Endogenous-NumberOfStocks: (1,)
world-Endogenous-StockPrice: (1,)
world-Endogenous-StockPriceHistory: (101,)
world-Endogenous-StocksLeft: (1,)
world-Endogenous-Supply: (1,)
world-Endogenous-TotalBalance: (1,)
world-Endogenous-Volumes: (101,)


[EnvWrapper] Obs (p)   
action_mask    : (1,)
time           : (1,)
world-Prices_History: (101,)
world-Total_Demand: (1,)
world-Total_Supply: (1,)
world-Volumes  : (101,)


[EnvWrapper] Action (a) Discrete(21)
[EnvWrapper] Action (p) Discrete(1)


In [5]:
import ray
from ray.rllib.agents.ppo import PPOTrainer

In [6]:
policies = {
    "a": (
        None,  # uses default policy
        env_obj.observation_space,
        env_obj.action_space,
        {}  # define a custom agent policy configuration.
    ),
    "p": (
        None,  # uses default policy
        env_obj.observation_space_pl,
        env_obj.action_space_pl,
        {}  # define a custom planner policy configuration.
    )
}

# In foundation, all the agents have integer ids and the social planner has an id of "p"
policy_mapping_fun = lambda i: "a" if str(i).isdigit() else "p"

policies_to_train = ["a", "p"]

In [7]:
trainer_config = {
    "multiagent": {
        "policies": policies,
        "policies_to_train": policies_to_train,
        "policy_mapping_fn": policy_mapping_fun,
    }
}

In [8]:
trainer_config.update(
    {
        "num_workers": 2,
        "num_envs_per_worker": 2,
        # Other training parameters
        "train_batch_size":  4000,
        "sgd_minibatch_size": 4000,
        "num_sgd_iter": 1
    }
)

In [9]:
# We also add the "num_envs_per_worker" parameter for the env. wrapper to index the environments.
env_config = {
    "env_config_dict": env_config_dict,
    "num_envs_per_worker": trainer_config.get('num_envs_per_worker'),   
}

trainer_config.update(
    {
        "env_config": env_config        
    }
)

In [10]:
# Initialize Ray
ray.init(webui_host="127.0.0.1")

2024-05-04 13:07:59,484	INFO resource_spec.py:212 -- Starting Ray with 5.18 GiB memory available for workers and up to 2.59 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2024-05-04 13:08:00,958	INFO services.py:1165 -- View the Ray dashboard at 127.0.0.1:8265


{'node_ip_address': '192.168.1.93',
 'raylet_ip_address': '192.168.1.93',
 'redis_address': '192.168.1.93:6379',
 'object_store_address': 'tcp://127.0.0.1:60366',
 'raylet_socket_name': 'tcp://127.0.0.1:58315',
 'webui_url': '127.0.0.1:8265',
 'session_dir': 'C:\\Users\\caspe\\AppData\\Local\\Temp\\ray\\session_2024-05-04_13-07-59_484645_2132'}

In [11]:
# Create the PPO trainer.
trainer = PPOTrainer(
    env=RLlibEnvWrapper,
    config=trainer_config,
    )

2024-05-04 13:08:06,896	ERROR syncer.py:46 -- Log sync requires rsync to be installed.
2024-05-04 13:08:06,909	INFO trainer.py:585 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
2024-05-04 13:08:06,909	INFO trainer.py:612 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


std  0.4838047165801802
liquidity 1.0
std  0.49555897723393394
liquidity 1.0
(pid=3720) C:\Users\caspe\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=3720)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=3720) C:\Users\caspe\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=3720)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=3720) C:\Users\caspe\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood a

(pid=3720) Inside covid19_components.py: 0 GPUs are available.
(pid=3720) No GPUs found! Running the simulation on a CPU.
(pid=3720) Inside covid19_env.py: 0 GPUs are available.
(pid=3720) No GPUs found! Running the simulation on a CPU.
(pid=1624) Inside covid19_components.py: 0 GPUs are available.
(pid=1624) No GPUs found! Running the simulation on a CPU.
(pid=1624) Inside covid19_env.py: 0 GPUs are available.
(pid=1624) No GPUs found! Running the simulation on a CPU.
(pid=3720) std  0.48581694058457014
(pid=3720) liquidity 1.0
(pid=1624) std  0.470899151261124
(pid=1624) liquidity 1.0


2024-05-04 13:08:18,501	INFO trainable.py:181 -- _setup took 11.605 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [12]:
NUM_ITERS = 5
start_train = time.time()

for iteration in range(NUM_ITERS):
    print(f'********** Iter : {iteration} **********')
    start = time.time()
    result = trainer.train()
    length = time.time() - start
    print(f'''episode_reward_mean: {result.get('episode_reward_mean')}''')
    print(f'''it_time_taken: {length}''')

    
length_train = time.time() - start_train
print("Training took", length_train, " seconds.")

********** Iter : 0 **********
(pid=3720) std  0.46159000641508785
(pid=3720) liquidity 1.0
(pid=1624) std  0.4626180423225086
(pid=1624) liquidity 1.0
(pid=3720) std  0.4766150614124369
(pid=3720) liquidity 1.0
(pid=1624) std  0.46651147125353765
(pid=1624) liquidity 1.0
(pid=3720) std  0.5092874674134616
(pid=3720) liquidity 1.0
(pid=1624) std  0.49694221934434357
(pid=1624) liquidity 1.0
(pid=3720) std  0.47478556000132155
(pid=3720) liquidity 1.0
(pid=3720) std  0.4753941218532036
(pid=3720) liquidity 0.0
(pid=3720) std  0.4319876347837238
(pid=3720) liquidity 1.0
(pid=3720) std  0.4518047808110348
(pid=3720) liquidity 1.0
(pid=3720) std  0.41282729263062645
(pid=3720) liquidity 1.0
(pid=3720) std  0.4402483645668903
(pid=3720) liquidity 1.0
(pid=3720) std  0.3806004387684012
(pid=3720) liquidity 1.0
(pid=3720) std  0.44964366488019547
(pid=3720) liquidity 1.0
(pid=3720) std  0.4258227084409858
(pid=3720) liquidity 1.0
(pid=3720) std  0.45948827588797647
(pid=3720) liquidity 1.0
(p

(pid=3720) std  1.0744614606570781
(pid=3720) liquidity 1.0
(pid=3720) std  1.1106308644106897
(pid=3720) liquidity 1.0
(pid=3720) std  1.0744614606570781
(pid=3720) liquidity 1.0
(pid=3720) std  1.1106308644106897
(pid=3720) liquidity 0.0
(pid=3720) std  1.0744614606570781
(pid=3720) liquidity 0.0
(pid=3720) std  1.1106308644106897
(pid=3720) liquidity 1.0
(pid=3720) std  1.0744614606570781
(pid=3720) liquidity 1.0
(pid=3720) std  1.1106308644106897
(pid=3720) liquidity 0.0
(pid=3720) std  1.0744614606570781
(pid=3720) liquidity 0.0
(pid=3720) std  1.1106308644106897
(pid=3720) liquidity 0.0
(pid=3720) std  1.0744614606570781
(pid=3720) liquidity 1.0
(pid=3720) std  1.1106308644106897
(pid=3720) liquidity 0.0
(pid=3720) std  1.0744614606570781
(pid=3720) liquidity 1.0
(pid=3720) std  1.1106308644106897
(pid=3720) liquidity 1.0
(pid=1624) std  1.0614817499043725
(pid=1624) liquidity 0.0
(pid=1624) std  1.076025145643463
(pid=1624) liquidity 1.0
(pid=1624) std  1.0614817499043725
(pid=1

(pid=3720) std  1.0744614606570781
(pid=3720) liquidity 1.0
(pid=3720) std  1.1106308644106897
(pid=3720) liquidity 1.0
(pid=3720) std  1.0744614606570781
(pid=3720) liquidity 0.0
(pid=3720) std  1.1106308644106897
(pid=3720) liquidity 0.0
(pid=3720) std  1.0744614606570781
(pid=3720) liquidity 0.0
(pid=3720) std  1.1106308644106897
(pid=3720) liquidity 1.0
(pid=3720) std  1.0744614606570781
(pid=3720) liquidity 0.0
(pid=3720) std  1.1106308644106897
(pid=3720) liquidity 1.0
(pid=3720) std  1.0744614606570781
(pid=3720) liquidity 1.0
(pid=3720) std  1.1106308644106897
(pid=3720) liquidity 0.0
(pid=3720) std  1.0744614606570781
(pid=3720) liquidity 0.0
(pid=3720) std  1.1106308644106897
(pid=3720) liquidity 1.0
(pid=1624) std  1.0614817499043725
(pid=1624) liquidity 0.0
(pid=1624) std  1.076025145643463
(pid=1624) liquidity 1.0
(pid=1624) std  1.0614817499043725
(pid=1624) liquidity 0.0
(pid=1624) std  1.076025145643463
(pid=1624) liquidity 1.0
(pid=1624) std  1.0614817499043725
(pid=16

(pid=3720) std  1.0535442223177232
(pid=3720) liquidity 1.0
(pid=3720) std  1.0255496382552727
(pid=3720) liquidity 1.0
(pid=3720) std  1.0535442223177232
(pid=3720) liquidity 1.0
(pid=3720) std  1.0255496382552727
(pid=3720) liquidity 1.0
(pid=3720) std  1.0535442223177232
(pid=3720) liquidity 1.0
(pid=3720) std  1.0255496382552727
(pid=3720) liquidity 1.0
(pid=3720) std  1.0535442223177232
(pid=3720) liquidity 1.0
(pid=3720) std  1.0255496382552727
(pid=3720) liquidity 1.0
(pid=3720) std  1.0535442223177232
(pid=3720) liquidity 1.0
(pid=3720) std  1.0255496382552727
(pid=3720) liquidity 0.0
(pid=3720) std  1.0535442223177232
(pid=3720) liquidity 0.0
(pid=3720) std  1.0255496382552727
(pid=3720) liquidity 1.0
(pid=3720) std  1.0535442223177232
(pid=3720) liquidity 0.0
(pid=1624) std  1.082166981383551
(pid=1624) liquidity 0.0
(pid=1624) std  1.0606170221522568
(pid=1624) liquidity 1.0
(pid=1624) std  1.082166981383551
(pid=1624) liquidity 1.0
(pid=1624) std  1.0606170221522568
(pid=16

(pid=3720) std  1.0535442223177232
(pid=3720) liquidity 0.0
(pid=1624) std  1.0606170221522568
(pid=1624) liquidity 0.0
(pid=3720) std  1.0255496382552727
(pid=3720) liquidity 1.0
(pid=1624) std  1.082166981383551
(pid=1624) liquidity 0.0
(pid=3720) std  1.0535442223177232
(pid=3720) liquidity 0.0
(pid=1624) std  1.0606170221522568
(pid=1624) liquidity 0.0
(pid=3720) std  1.0255496382552727
(pid=3720) liquidity 0.0
(pid=3720) std  1.0535442223177232
(pid=3720) liquidity 1.0
(pid=1624) std  1.082166981383551
(pid=1624) liquidity 0.0
(pid=3720) std  1.0255496382552727
(pid=3720) liquidity 1.0
(pid=1624) std  1.0606170221522568
(pid=1624) liquidity 1.0
(pid=3720) std  1.0535442223177232
(pid=3720) liquidity 1.0
(pid=3720) std  1.0255496382552727
(pid=3720) liquidity 0.0
(pid=3720) std  1.0535442223177232
(pid=3720) liquidity 0.0
(pid=3720) std  1.0255496382552727
(pid=3720) liquidity 0.0
(pid=3720) std  1.0535442223177232
(pid=3720) liquidity 0.0
(pid=3720) std  1.0255496382552727
(pid=37

...

(pid=1624) std  0.46140456378714745
(pid=1624) liquidity 1.0
(pid=3720) std  0.45659141524360736
(pid=3720) liquidity 1.0
(pid=3720) std  0.4685917624161121
(pid=3720) liquidity 1.0
(pid=3720) std  0.410514138078666
(pid=3720) liquidity 1.0
(pid=3720) std  0.45371627262645375
(pid=3720) liquidity 1.0
(pid=1624) std  0.4829181418405696
(pid=1624) liquidity 1.0
(pid=1624) std  0.42267519114948326
(pid=1624) liquidity 1.0
(pid=1624) std  0.4363744718736875
(pid=1624) liquidity 1.0
(pid=1624) std  0.3945705334926626
(pid=1624) liquidity 1.0
(pid=3720) std  0.3672880021629933
(pid=3720) liquidity 1.0
(pid=3720) std  0.4044532452140819
(pid=3720) liquidity 1.0
(pid=1624) std  0.4450168090316614
(pid=1624) liquidity 1.0
(pid=3720) std  0.3509103594460796
(pid=3720) liquidity 1.0
(pid=3720) std  0.4067325589022881
(pid=3720) liquidity 1.0
(pid=3720) std  0.3122645649740198
(pid=3720) liquidity 1.0
(pid=3720) std  0.38034332563002204
(pid=3720) liquidity 1.0
(pid=3720) std  0.2312039502650619
(

(pid=3720) std  1.0468069868233578
(pid=3720) liquidity 1.0
(pid=3720) std  1.0912552171986074
(pid=3720) liquidity 0.0
(pid=3720) std  1.0468069868233578
(pid=3720) liquidity 0.0
(pid=3720) std  1.0912552171986074
(pid=3720) liquidity 0.0
(pid=3720) std  1.0468069868233578
(pid=3720) liquidity 0.0
(pid=3720) std  1.0912552171986074
(pid=3720) liquidity 1.0
(pid=3720) std  1.0468069868233578
(pid=3720) liquidity 1.0
(pid=3720) std  1.0912552171986074
(pid=3720) liquidity 0.0
(pid=3720) std  1.0468069868233578
(pid=3720) liquidity 0.0
(pid=3720) std  1.0912552171986074
(pid=3720) liquidity 0.0
(pid=3720) std  1.0468069868233578
(pid=3720) liquidity 1.0
(pid=3720) std  1.0912552171986074
(pid=3720) liquidity 1.0
(pid=1624) std  1.0767411962251296
(pid=1624) liquidity 1.0
(pid=1624) std  1.0764204537143431
(pid=1624) liquidity 0.0
(pid=1624) std  1.0767411962251296
(pid=1624) liquidity 1.0
(pid=1624) std  1.0764204537143431
(pid=1624) liquidity 0.0
(pid=1624) std  1.0767411962251296
(pid=

(pid=3720) liquidity 1.0
(pid=1624) std  1.0764204537143431
(pid=1624) liquidity 1.0
(pid=3720) std  1.0468069868233578
(pid=3720) liquidity 0.0
(pid=3720) std  1.0912552171986074
(pid=3720) liquidity 0.0
(pid=3720) std  1.0468069868233578
(pid=3720) liquidity 0.0
(pid=3720) std  1.0912552171986074
(pid=3720) liquidity 0.0
(pid=3720) std  1.0468069868233578
(pid=3720) liquidity 0.0
(pid=3720) std  1.0912552171986074
(pid=3720) liquidity 0.0
(pid=3720) std  1.0468069868233578
(pid=3720) liquidity 0.0
(pid=3720) std  1.0912552171986074
(pid=3720) liquidity 0.0
(pid=1624) std  1.0767411962251296
(pid=1624) liquidity 0.0
(pid=1624) std  1.0764204537143431
(pid=1624) liquidity 1.0
(pid=1624) std  1.0767411962251296
(pid=1624) liquidity 0.0
(pid=1624) std  1.0764204537143431
(pid=1624) liquidity 1.0
(pid=1624) std  1.0767411962251296
(pid=1624) liquidity 0.0
(pid=1624) std  1.0764204537143431
(pid=1624) liquidity 1.0
(pid=1624) std  1.0767411962251296
(pid=1624) liquidity 0.0
(pid=1624) std 

KeyboardInterrupt: 

(pid=3720) std  1.0468069868233578
(pid=3720) liquidity 0.0
(pid=3720) std  1.0912552171986074
(pid=3720) liquidity 1.0
(pid=3720) std  1.0468069868233578
(pid=3720) liquidity 0.0
(pid=3720) std  1.0912552171986074
(pid=3720) liquidity 0.0
(pid=3720) std  1.0468069868233578
(pid=3720) liquidity 0.0
(pid=3720) std  1.0912552171986074
(pid=3720) liquidity 0.0
(pid=3720) std  1.0468069868233578
(pid=3720) liquidity 0.0
(pid=3720) std  1.0912552171986074
(pid=3720) liquidity 0.0
(pid=3720) std  1.0468069868233578
(pid=3720) liquidity 0.0
(pid=3720) std  1.0912552171986074
(pid=3720) liquidity 0.0
(pid=3720) std  1.0468069868233578
(pid=3720) liquidity 1.0
(pid=3720) std  1.0912552171986074
(pid=3720) liquidity 0.0
(pid=3720) std  1.0468069868233578
(pid=3720) liquidity 0.0
(pid=3720) std  1.0912552171986074
(pid=3720) liquidity 1.0
(pid=1624) std  1.0767411962251296
(pid=1624) liquidity 1.0
(pid=1624) std  1.0764204537143431
(pid=1624) liquidity 0.0
(pid=1624) std  1.0767411962251296
(pid=

In [ ]:
def generate_rollout_from_current_trainer_policy(
    trainer, 
    env_obj,
    num_dense_logs=1
):
    dense_logs = {}
    for idx in range(num_dense_logs):
        # Set initial states
        agent_states = {}
        for agent_idx in range(env_obj.env.n_agents):
            agent_states[str(agent_idx)] = trainer.get_policy("a").get_initial_state()
        planner_states = trainer.get_policy("p").get_initial_state()   

        # Play out the episode
        obs = env_obj.reset(force_dense_logging=True)
        for t in range(env_obj.env.episode_length):
            actions = {}
            for agent_idx in range(env_obj.env.n_agents):
                # Use the trainer object directly to sample actions for each agent
                actions[str(agent_idx)] = trainer.compute_action(
                    obs[str(agent_idx)], 
                    agent_states[str(agent_idx)], 
                    policy_id="a",
                    full_fetch=False
                )

            # Action sampling for the planner
            actions["p"] = trainer.compute_action(
                obs['p'], 
                planner_states, 
                policy_id='p',
                full_fetch=False
            )

            obs, rew, done, info = env_obj.step(actions)        
            if done['__all__']:
                break
        dense_logs[idx] = env_obj.env.dense_log
    return dense_logs

In [ ]:
dense_logs = generate_rollout_from_current_trainer_policy(
    trainer, 
    env_obj,
    num_dense_logs=1
)

In [ ]:
agent_0_total_balance = [log["0"]["endogenous"]["TotalBalance"] for log in dense_logs[0]['states']]
agent_1_total_balance = [log["1"]["endogenous"]["TotalBalance"] for log in dense_logs[0]['states']]
agent_2_total_balance = [log["2"]["endogenous"]["TotalBalance"] for log in dense_logs[0]['states']]
agent_3_total_balance = [log["3"]["endogenous"]["TotalBalance"] for log in dense_logs[0]['states']]

stock_prices = [log["0"]["endogenous"]["StockPrice"] for log in dense_logs[0]['states']]

range = np.arange(0, 100 + 1, 2.0)

In [ ]:
plt.figure(figsize=(16, 6))
plt.plot(np.arange(0,101),agent_0_total_balance, label="Agent 0")
plt.plot(np.arange(0,101),agent_1_total_balance, label="Agent 1")
plt.plot(np.arange(0,101),agent_2_total_balance, label="Agent 2")
plt.plot(np.arange(0,101),agent_3_total_balance, label="Agent 3")
plt.title('Stock Broker Total Balance')
plt.legend()
plt.xlabel('Days')
plt.ylabel('Total Balance')
plt.xticks(range)

# Adding vertical lines at each xtick position
for xtick in range:
    plt.axvline(x=xtick, color='gray', linestyle='--', linewidth=0.5)
    
#plt.savefig("miners_green_scores.png")
plt.show()

In [ ]:
plt.figure(figsize=(16, 6))
plt.plot(np.arange(0, 101), stock_prices, label="Stock Price")
plt.title('Stock Price Over Time')
plt.legend()
plt.xlabel('Days')
plt.ylabel('Stock Price')
plt.xticks(range)

# Adding vertical lines at each xtick position
for xtick in range:
    plt.axvline(x=xtick, color='gray', linestyle='--', linewidth=0.5)
    
plt.show()

In [ ]:
volumes = [log["0"]["endogenous"]["Volumes"] for log in dense_logs[0]['states']]
prices = [log["0"]["endogenous"]["StockPriceHistory"] for log in dense_logs[0]['states']]

agent_0_demand = [log["0"]["endogenous"]["Demand"] for log in dense_logs[0]['states']]
agent_1_demand = [log["1"]["endogenous"]["Demand"] for log in dense_logs[0]['states']]
agent_2_demand = [log["2"]["endogenous"]["Demand"] for log in dense_logs[0]['states']]
agent_3_demand = [log["3"]["endogenous"]["Demand"] for log in dense_logs[0]['states']]

agent_0_supply = [log["0"]["endogenous"]["Supply"] for log in dense_logs[0]['states']]
agent_1_supply = [log["1"]["endogenous"]["Supply"] for log in dense_logs[0]['states']]
agent_2_supply = [log["2"]["endogenous"]["Supply"] for log in dense_logs[0]['states']]
agent_3_supply = [log["3"]["endogenous"]["Supply"] for log in dense_logs[0]['states']]

agents_demands = [x + y + z + f for x, y, z, f in zip(agent_0_demand, agent_1_demand,agent_2_demand, agent_3_demand )]
agents_supplies = [x + y + z + f for x, y, z, f in zip(agent_0_supply, agent_1_supply,agent_2_supply, agent_3_supply )]

i=0
while i < 101:
    print("--- Summary of Day", i, "---")
    print("Price: ", prices[-1][i])
    print("Volume: ", volumes[-1][i])
    print("Demand: ", agents_demands[i], "  Supply: ", agents_supplies[i])
    demand = agents_demands[i]
    supply = agents_supplies[i]
    if demand+supply>0:
        test = (demand - supply) / (demand + supply)
        test = 0.05*test
        #print(test)
    else:
        pass
        #print(0)
    print("")
    i+=1



In [ ]:
stocks_left = [log["0"]["endogenous"]["StocksLeft"] for log in dense_logs[0]['states']]

plt.plot(np.arange(0,101),stocks_left, label="Stocks Quantity Left")
plt.title('Stock Quantity Over Time')
plt.legend()
plt.xlabel('Days')
plt.ylabel('Stock Price')
plt.xticks(np.arange(0, 100+1, 5.0))
#plt.savefig("miners_green_scores.png")
plt.show()

In [ ]:
# Shutdown Ray after use
ray.shutdown()

In [ ]:
#planner_gr_score_importances = [log["p"]["GreenScoreImportance"] for log in dense_logs[0]['states']]

In [ ]:
#dense_logs[0]['states'][0]